In [1]:
import csv
import time

start = time.time()

with open('wifi_list.csv') as f:
    wifiTable = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

print "Data Import : " + str(time.time() - start)

Import : 0.0930860042572


In [11]:
## Truncate the table 
conn = pymysql.connect(host='mysql.clarksonmsda.org', \
                       port=3306, \
                       user='ia626', \
                       passwd='ia626clarkson', \
                       db='ia626', autocommit=True)
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("TRUNCATE TABLE `mdhaque_wifi`")
print "Table Truncate : " + str(time.time() - start)

Table Truncate : 177.265734911


In [12]:
## Buffered insert to insert faster
i=0
sql=''
tokens = []
n= 0

sql = '''INSERT INTO mdhaque_wifi (`MAC`,`SSID`,`AuthMode`,`FirstSeen`, 
    `Channel`,`RSSI`,`CurrentLatitude`,`CurrentLongitude`,`AltitudeMeters`,`Type`
    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
blocksizes = [1000]
for bs in blocksizes:
    start = time.time()
    for row in wifiTable:
        dtl = row['FirstSeen'].split(' ')
        dts = '0000-00-00 00:00'
        if len(dtl) == 2:
            dl = dtl[0].split("/")
            dts = dl[2] + '/' + dl[0] + '/'+dl[1] + ' ' +dtl[1] 
        tokens.append((row["MAC"],row["SSID"],row["AuthMode"],dts, \
        row["Channel"],row["RSSI"],row["CurrentLatitude"],row["CurrentLongitude"],\
        row["AltitudeMeters"],row["Type"]))
        #print conn.insert_id()
        if i % bs == 0:
            n+=1
            #print tokens
            bstart = time.time()
            cur.executemany(sql,tokens)
            conn.commit()
            #print "Block " + str(n) +" "+ str(time.time() - bstart)
            tokens = []
        i+=1
    print "block size: " + str(bs) + " - total time : " + str(time.time() - start)
    if len(tokens) > 0:
        cur.executemany(sql,tokens)
        conn.commit()
    
cur.close()
conn.close()

block size: 1000 - total time : 1.63105893135


In [2]:
!head wifi_list.csv

MAC,SSID,AuthMode,FirstSeen,Channel,RSSI,CurrentLatitude,CurrentLongitude,AltitudeMeters,AccuracyMeters,Type
e0:91:f5:de:fa:4f,NETGEAR-5G,[WPS][ESS],5/9/2012 2:02,36,-67,44.67023722,-74.98363701,104.1999969,7,WIFI
e2:91:f5:de:fa:51,bonesaw_Guest,[WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][ESS],5/9/2012 2:02,11,-62,44.67023722,-74.98363701,104.1999969,7,WIFI
e0:91:f5:de:fa:50,BONESAW_HQ,[WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][WPS][ESS],5/9/2012 2:02,11,-62,44.67023722,-74.98363701,104.1999969,7,WIFI
e0:91:f5:5e:34:60,Happy_Clam,[WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][WPS][ESS],5/9/2012 2:02,6,-61,44.67023722,-74.98363701,104.1999969,7,WIFI
90:84:0d:d6:8b:69,Nationwide,[WPA2-PSK-CCMP][ESS],5/9/2012 2:02,1,-85,44.67023722,-74.98363701,104.1999969,7,WIFI
00:08:9f:7b:06:d1,myLGNet,[WEP][ESS],5/9/2012 2:02,11,-79,44.67023722,-74.98363701,104.1999969,7,WIFI
00:23:69:c1:a9:ef,linksys,[WPA-PSK-TKIP][ESS],5/9/2012 2:02,6,-87,44.67023722,-74.98363701,104.1999969,7,WIFI
310410_11506_144930464,AT&T,UMT

In [7]:
import pymysql # run pip install pymysql if this fails
import csv
import time

start = time.time()

with open('wifi_list.csv') as f:
    wifiTable = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

print "Import : " + str(time.time() - start)

conn = pymysql.connect(host='mysql.clarksonmsda.org', port=3306, user='ia626', \
                       passwd='ia626clarkson', db='ia626', autocommit=True) #setup our credentials
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("TRUNCATE TABLE `mdhaque_wifi`")
i=0
tokens = []
bs = 500
n= 0

sql = '''INSERT INTO mdhaque_wifi (`MAC`,`SSID`,`AuthMode`,`FirstSeen`, 
    `Channel`,`RSSI`,`CurrentLatitude`,`CurrentLongitude`,`AltitudeMeters`, 
    `Type`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
start = time.time()
for row in wifiTable:
    '''
    [['a','b','c'],['a1','b1','c1'],['a2','b2','c2']]
    '''
    dtl = row['FirstSeen'].split(' ')
    dts = '0000-00-00 00:00'
    if len(dtl) == 2:
        dl = dtl[0].split("/")
        dts = dl[2] + '/' + dl[0] + '/'+dl[1] + ' ' +dtl[1] 
    tokens.append((row["MAC"],row["SSID"],row["AuthMode"],dts, \
    row["Channel"],row["RSSI"],row["CurrentLatitude"],row["CurrentLongitude"],\
    row["AltitudeMeters"],row["Type"]))
    #print conn.insert_id()
    if i % bs == 0:
        n+=1
        #print tokens
        bstart = time.time()
        cur.executemany(sql,tokens)
        conn.commit()
        print "Block " + str(n) +" "+ str(time.time() - bstart)
        tokens = []
    i+=1
print "block size: " + str(bs) + " - total time : " + str(time.time() - start)
if len(tokens) > 0:
    cur.executemany(sql,tokens)
    conn.commit()
    
cur.close()
conn.close()

Import : 0.148526906967
Block 1 0.0334811210632
Block 2 0.164397001266
Block 3 0.12668800354
Block 4 0.116857051849
Block 5 0.123313903809
Block 6 0.109452962875
Block 7 0.11803984642
Block 8 0.14015007019
Block 9 0.120574951172
block size: 500 - total time : 1.10730409622


In [6]:
## At the begining if the file does not exist CREATE TABLE
import pymysql # run pip install pymysql if this fails

conn = pymysql.connect(host='mysql.clarksonmsda.org', \
                       port=3306, \
                       user='ia626', \
                       passwd='ia626clarkson', \
                       db='ia626', autocommit=True) #setup credential from config file??
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute('''

CREATE TABLE IF NOT EXISTS `mdhaque_wifi` (
  `id` int(5) NOT NULL AUTO_INCREMENT,
  `MAC` varchar(50) NOT NULL,
  `SSID` varchar(50) NOT NULL,
  `AuthMode` varchar(100) NOT NULL,
  `FirstSeen` datetime NOT NULL,
  `Channel` int(3) NOT NULL,
  `RSSI` int(5) NOT NULL,
  `CurrentLatitude` decimal(11,8) NOT NULL,
  `CurrentLongitude` decimal(11,8) NOT NULL,
  `AltitudeMeters` decimal(15,10) NOT NULL,
  `Type` varchar(50) NOT NULL,
  PRIMARY KEY (`id`)
) ENGINE=MyISAM  DEFAULT CHARSET=latin1 AUTO_INCREMENT=4485 ;
''')

cur.close()
conn.close()